In [1]:
#All the required required modules to build a recommendation system for the given data. 
import pandas as pd 
import numpy as np
import re
import string
import enchant
from googletrans import Translator
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
movies_data = pd.read_csv('movies_metadata.csv', header = 0)

C:\Users\Vinay\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
movies_data.columns# Column names or feature names

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

Now getting required columns to build  the recommendation system. 

In [4]:
data = movies_data[['title', 'original_title', 'tagline', 'overview', 'genres']]

In [5]:
print('Number of data points: {0} , Number of Features: {1}'.format(data.shape[0], data.shape[1]))
data.head()

Number of data points: 45466 , Number of Features: 5


,title,original_title,tagline,overview,genres
0,Toy Story,Toy Story,NaN,"Led by Woody, Andy's toys live happily in his ...","[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
1,Jumanji,Jumanji,Roll the dice and unleash the excitement!,When siblings Judy and Peter discover an encha...,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,Grumpier Old Men,Grumpier Old Men,Still Yelling. Still Fighting. Still Ready for...,A family wedding reignites the ancient feud be...,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
3,Waiting to Exhale,Waiting to Exhale,Friends are the people who let you be yourself...,"Cheated on, mistreated and stepped on, the wom...","[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
4,Father of the Bride Part II,Father of the Bride Part II,Just When His World Is Back To Normal... He's ...,Just when George Banks has recovered from his ...,"[{'id': 35, 'name': 'Comedy'}]"


In [6]:
# Print column names or feature names Total of 4 features. 
data.columns

Index(['title', 'original_title', 'tagline', 'overview', 'genres'], dtype='object')

In [7]:
print(sum(data['title'].value_counts().to_frame('count')['count'] > 1))
print(data['title'].value_counts())
#There are total of 2330 duplicate titles.

2330
Cinderella                          11
Alice in Wonderland                  9
Hamlet                               9
Beauty and the Beast                 8
Les Misérables                       8
                                    ..
Kids                                 1
Stepmom                              1
Robin Williams: Live on Broadway     1
Night Will Fall                      1
Welcome to Hard Times                1
Name: title, Length: 42277, dtype: int64


In [8]:
data.drop_duplicates(subset = 'title', keep = 'first' , inplace = True)
data.reset_index(drop = True, inplace = True)

C:\Users\Vinay\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [9]:
data.head()

,title,original_title,tagline,overview,genres
0,Toy Story,Toy Story,NaN,"Led by Woody, Andy's toys live happily in his ...","[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
1,Jumanji,Jumanji,Roll the dice and unleash the excitement!,When siblings Judy and Peter discover an encha...,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,Grumpier Old Men,Grumpier Old Men,Still Yelling. Still Fighting. Still Ready for...,A family wedding reignites the ancient feud be...,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
3,Waiting to Exhale,Waiting to Exhale,Friends are the people who let you be yourself...,"Cheated on, mistreated and stepped on, the wom...","[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
4,Father of the Bride Part II,Father of the Bride Part II,Just When His World Is Back To Normal... He's ...,Just when George Banks has recovered from his ...,"[{'id': 35, 'name': 'Comedy'}]"


In [10]:
for i in range(len(data)):
    try:
        data['title'][i] = data['title'][i].lower()
        data['original_title'][i] = data['original_title'][i].lower()
    except:
        print('Cannot be convert into lower_case:{}'.format(data['title'][i]))
        print('Cannot be convert into lower_case:{}'.format(data['original_title'][i]))

C:\Users\Vinay\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3331: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


Cannot be convert into lower_case:nan
Cannot be convert into lower_case:Midnight Man


In [11]:
print(sum(data['title'].value_counts().to_frame('count')['count'] > 1))

50


In [12]:
data.drop_duplicates(subset = 'title', keep = 'first' , inplace = True)
data.reset_index(drop = True, inplace = True)

C:\Users\Vinay\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [13]:
print('Previously it was: 45466 but after removing duplicates Number of data points: {0} '.format(data.shape[0]))
data.sort_values('title', inplace = True, ascending = True)# Sort the dataset with title. 
data.reset_index(drop = True, inplace = True)
data.head()

Previously it was: 45466 but after removing duplicates Number of data points: 42228 


C:\Users\Vinay\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,title,original_title,tagline,overview,genres
0,!women art revolution,!women art revolution,40 years in the making!,"Through intimate interviews, provocative art, ...","[{'id': 99, 'name': 'Documentary'}]"
1,#1 cheerleader camp,#1 cheerleader camp,Grab your spirit sticks!,A pair of horny college guys get summer jobs a...,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
2,#chicagogirl,#chicagogirl,The Social Network Takes on a Dictator,From her childhood bedroom in the Chicago subu...,"[{'id': 99, 'name': 'Documentary'}]"
3,#horror,#horror,Death is trending.,"Inspired by actual events, a group of 12 year ...","[{'id': 18, 'name': 'Drama'}, {'id': 9648, 'na..."
4,"$1,000 on the black",mille dollari sul nero,NaN,Johnny Liston has just been released from pris...,"[{'id': 37, 'name': 'Western'}]"


In [14]:
data.tail()

,title,original_title,tagline,overview,genres
42223,‘rameau’s nephew’ by diderot (thanx to dennis ...,‘rameau’s nephew’ by diderot (thanx to dennis ...,NaN,Described (rather cheekily) by director Michae...,[]
42224,’round midnight,’round midnight,The music and the magic come together...,Inside the Blue Note nightclub one night in 19...,"[{'id': 18, 'name': 'Drama'}]"
42225,…and the fifth horseman is fear,…a pátý jezdec je strach,NaN,A Jewish doctor helps a political fugitive dur...,"[{'id': 18, 'name': 'Drama'}, {'id': 10752, 'n..."
42226,ファンタスティポ,ファンタスティポ,NaN,Two brothers must take the helm of their famil...,"[{'id': 35, 'name': 'Comedy'}]"
42227,NaN,Midnight Man,NaN,British soldiers force a recently captured IRA...,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam..."


In [15]:
data['title'].isin(data['original_title']).value_counts()

True     31850
False    10378
Name: title, dtype: int64

In [16]:
translate_title = []
for i in range(len(data)):
    if data['title'][i] != data['original_title'][i]:
        translate_title.append(data['original_title'][i])

In [17]:
translate_title

['mille dollari sul nero',
 'centomila dollari per ringo',
 'feng ai',
 '(entlastungen) pipilottis fehler',
 'et dieu… créa la femme',
 'calibre 45',
 'a 008, operazione sterminio',
 '009 re：cyborg',
 '009ノ1（ゼロゼロクノイチ） the end of the beginning',
 '1 - నేనొక్కడినే',
 '１リットルの涙',
 '06',
 '10 milliarden - wie werden wir alle satt?',
 '10 minuta',
 '10.000 dollari per un massacro',
 '10.000 km',
 '내 사랑 싸가지',
 '100 gramm dlya khrabrosti',
 '100 metros',
 '殺妻二人組',
 '100% లవ్',
 '1001 gram',
 'les 1001 nuits',
 '101 reykjavík',
 '1066',
 '열한시',
 '我11',
 '11 minut',
 '12 стульев',
 '12 meter ohne kopf',
 '海難1890',
 'a fost sau n-a fost?',
 '十三人の刺客',
 '13 เกมสยอง',
 'trece campanadas',
 '13 dies de octubre',
 'elser',
 'jin yi wei',
 '15 años y un día',
 'la fille de brest',
 '1612: хроники смутного времени',
 '17 filles',
 '18 comidas',
 'diciotto anni dopo',
 "j'aime regarder les filles",
 '180° south',
 '1898. los últimos de filipinas',
 'novecento',
 '辛亥革命',
 'tali-ihantala 1944',
 '1990: i g

In [13]:
import time
translator = Translator()

In [19]:
translator.translate('10 milliarden - wie werden wir alle satt?').text

'10 billion - as we are all tired?'

In [20]:
data.isna().sum()

title                 1
original_title        0
tagline           23190
overview            914
genres                0
dtype: int64

In [21]:
data['title'].fillna(0, inplace = True)

C:\Users\Vinay\anaconda3\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [22]:
#I see the there's an NA value in Title which is not accepted. So I checked if there's any missing in original title and found
#that the missing one is of english version. So I moved the original title and filled it to the title row. 
# there would be no NA in title column now 
for i in range(len(data)):
    if data['title'][i] == 0:
        data['title'][i] = data['original_title'][i]

C:\Users\Vinay\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3331: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [23]:
d = enchant.Dict("en_US")

In [24]:
data['title'][4].split()

['$1,000', 'on', 'the', 'black']

In [25]:
string = data['title'][4].split()

In [26]:
for i in string:
    if d.check(i) == True:
        print(i)
    else:
        print('false {}'.format(i))

false $1,000
on
the
black


In [27]:
d.check('ファンタスティポ')

False

In [28]:
''.join(e for e in string if e.isalnum() or e.isdecimal())

'ontheblack'

In [29]:
data.shape

(42228, 5)

In [2]:
import pickle

In [31]:
# with open('data_movie.pickle', 'wb') as f:
#     pickle.dump(data, f)

pickle_in = open("data_movie.pickle","rb")
data = pickle.load(pickle_in)

In [6]:
import enchant
d = enchant.Dict("en_US")

In [34]:
# %%time
# for i in range(len(data)):
#     str_lst = data['title'][i].split()
#     for j in range(len(str_lst)):
#         string = str_lst[j]
#         x = ''.join(e for e in string if e.isalnum() or e.isdecimal())
#         try:
#             if d.check(x) == False:
#                 try:
#                     z = translator.translate(x, dest='en')
#                     if z.text != y:
#                         str_lst[j] = z.text
#                         data['title'][i] = ' '.join(sent for sent in str_lst)
#                 except:
#                     print(data['title'][i])
#                     print(x)
#         except:
#             continue

In [17]:
#dumped the data that has been done till here. 
with open('data_movie.pickle', 'wb') as f:
    pickle.dump(data, f)

In [156]:
#read the data. 
pickle_in = open("data_movie.pickle","rb")
data = pickle.load(pickle_in)

In [157]:
data.shape

(42228, 5)

In [158]:
#Getting NA values.
#Found NA in tagline and overview. 
#Title had NA too but I replaced it with original title. 
data.isna().sum()

title                 0
original_title        0
tagline           23190
overview            914
genres                0
dtype: int64

In [159]:
data.head()

,title,original_title,tagline,overview,genres
0,!women art revolution,!women art revolution,40 years in the making!,"Through intimate interviews, provocative art, ...","[{'id': 99, 'name': 'Documentary'}]"
1,#1 cheerleader camp,#1 cheerleader camp,Grab your spirit sticks!,A pair of horny college guys get summer jobs a...,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
2,#chicagogirl,#chicagogirl,The Social Network Takes on a Dictator,From her childhood bedroom in the Chicago subu...,"[{'id': 99, 'name': 'Documentary'}]"
3,#horror,#horror,Death is trending.,"Inspired by actual events, a group of 12 year ...","[{'id': 18, 'name': 'Drama'}, {'id': 9648, 'na..."
4,"$1,000 on the black",mille dollari sul nero,NaN,Johnny Liston has just been released from pris...,"[{'id': 37, 'name': 'Western'}]"


In [165]:
#Replacing all the na values from tagline and overview with never provided. 
data['tagline'].fillna('Never Provided', inplace = True)
data['overview'].fillna('Never Provided', inplace = True)

In [166]:
data.isna().sum()

title             0
original_title    0
tagline           0
overview          0
genres            0
dtype: int64

In [167]:
#created a function so that it can be useful for me in future. 
#this is only for tagline and overview.
#We can remove punctuations from the tagline and overview but I have not done it on title so that I can get exact titles and 
#genre informaiton when I run the recommendation system. 
#Prepcrossing methods  - Lowering, removing characters or punctuations and stop word in next few cells. 
def column_Preprocess_sentence(df,column):
    for index,row in df.iterrows():
        #print(index)
        word = row[column]
        word = ''.join([i for i in word if not i.isdigit()])
        word = word.lower()
        word = word.translate(str.maketrans('', '', string.punctuation))
        word = ' '.join(word.split())
        df[column][index] = word
    return('Prepcoessing done for:{}'.format(column))

In [169]:
%%time
column_Preprocess(data,'tagline')#Tagline preprocess

Wall time: 27.3 s


'Prepcoessing done for:tagline'

In [170]:
%%time
column_Preprocess(data,'overview')#overview preprocess

Wall time: 35 s


'Prepcoessing done for:overview'

In [172]:
#Stop word removal from the tagline and overview, created a function for that. 
def stop_words_remove(df, column):
    for index, row in df.iterrows():
        text_tokens = word_tokenize(row[column])
        tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]
        filtered_sentence = (" ").join(tokens_without_sw)
        df[column][index] = filtered_sentence
    return('All done for {}'.format(column))

In [175]:
%%time
stop_words_remove(data, 'tagline')#stop word tagline removed

Wall time: 32min 18s


'All done for tagline'

In [176]:
%%time
stop_words_remove(data, 'overview')#stop word overview removed. 

Wall time: 5h 57min 21s


'All done for overview'

In [177]:
data.head()

,title,original_title,tagline,overview,genres
0,!women art revolution,!women art revolution,years making,intimate interviews provocative art rare histo...,"[{'id': 99, 'name': 'Documentary'}]"
1,#1 cheerleader camp,#1 cheerleader camp,grab spirit sticks,pair horny college guys get summer jobs sexy c...,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
2,#chicagogirl,#chicagogirl,social network takes dictator,childhood bedroom chicago suburbs american tee...,"[{'id': 99, 'name': 'Documentary'}]"
3,#horror,#horror,death trending,inspired actual events group year old girls ni...,"[{'id': 18, 'name': 'Drama'}, {'id': 9648, 'na..."
4,"$1,000 on the black",mille dollari sul nero,never provided,johnny liston released prison wrongly served t...,"[{'id': 37, 'name': 'Western'}]"


In [178]:
#Preprocessing for title column 
def column_Preprocess_titile(df,column):
    for index,row in df.iterrows():
        #print(index)
        word = row[column]
       #word = ''.join([i for i in word if not i.isdigit()])
        #word = word.lower() 
        word = word.translate(str.maketrans('', '', string.punctuation))
        word = ' '.join(word.split())
        row[column] = word
    return('Prepcoessing done for:{}'.format(column))

In [179]:
column_Preprocess_titile(data,'title') # Prepcoessing title 

'Prepcoessing done for:title'

In [183]:
column_Preprocess_titile(data,'original_title') # Prepcoessing original_title 

'Prepcoessing done for:original_title'

In [184]:
data.head()

,title,original_title,tagline,overview,genres
0,women art revolution,women art revolution,years making,intimate interviews provocative art rare histo...,"[{'id': 99, 'name': 'Documentary'}]"
1,1 cheerleader camp,1 cheerleader camp,grab spirit sticks,pair horny college guys get summer jobs sexy c...,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
2,chicagogirl,chicagogirl,social network takes dictator,childhood bedroom chicago suburbs american tee...,"[{'id': 99, 'name': 'Documentary'}]"
3,horror,horror,death trending,inspired actual events group year old girls ni...,"[{'id': 18, 'name': 'Drama'}, {'id': 9648, 'na..."
4,1000 on the black,mille dollari sul nero,never provided,johnny liston released prison wrongly served t...,"[{'id': 37, 'name': 'Western'}]"


In [185]:
import ast 

In [186]:
#Genres column is in string format, using ast module to convert string to list.
for i,j in data.iterrows():
    data['genres'][i] = ast.literal_eval(data['genres'][i])

In [187]:
%%time
#Now, As I need the genres names, I wrote a loop to get them in a string format. 

for index,row in data.iterrows():
    gnre_ls = []
    if len(data['genres'][index]) <= 1: # If there's only one genre in that row.  
        for i in data['genres'][index]:
            data['genres'][index] = i['name']
    else:
        for i in data['genres'][index]:# If there's more then one genre name in that row
            gnre_ls.append(i['name'])
            genres = (" ").join(gnre_ls)
        data['genres'][index] = genres

Wall time: 34 s


In [188]:
%%time
#there are few genres where there's and empty list, so replacing thos list with None. Which mean, for this particular title
#there's no genre
c=0
for index, row in data.iterrows():
    if type(data['genres'][index]) == list:
        c+=1
        data['genres'][index] = 'None' 
print(c)

2277
Wall time: 7.65 s


In [189]:
#creating a column index
index = list(range(len(data)))
data['index'] = index

In [190]:
with open('data_movie.pickle', 'wb') as f:
    pickle.dump(data, f)

In [192]:
#I have the preprocessing which is required for this dataset

# Now loaded Tfidf vectorizer to covert Genre column to a vector. 
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(data['genres'])

In [193]:
#Cosine similarity - using this I can create a kernel. 
# Using cosine similarity, I can get related or close feautes/items/movies as recommendations. 
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [198]:
# Build a 1-dimensional array with movie titles
titles = data[['title','genres']]
indices = pd.Series(data.index, index=data['title'])

# Function that get movie recommendations based on the cosine similarity score of movie genres
def genre_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:21]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [199]:
#I called movies realted to Toy story and pops up related titles that a user can prefer. 
genre_recommendations('toy story')

,title,genres
837,a flintstones christmas carol,Animation Comedy Family
1083,a matter of loaf and death,Animation Comedy Family
2341,amazon jack 3 jungo goes bananas,Animation Comedy Family
2759,anina,Animation Comedy Family
3739,banana,Animation Comedy Family
3866,barnyard,Animation Comedy Family
3886,bartok the magnificent,Animation Comedy Family
4576,big top scoobydoo,Animation Comedy Family
5254,bon voyage charlie brown and dont come back,Animation Comedy Family
5363,botsman i popugay,Animation Comedy Family


In [201]:
#checking if recommendation systems and provided details with respective to the title genre. 
for index,row in data.iterrows():
    if row['title'] == 'toy story':
        print(data['genres'][index])

Animation Comedy Family


In [202]:
#I called movies realted to Toy story and pops up related titles that a user can prefer. 
genre_recommendations('a chinese ghost story ii')

,title,genres
12730,ghoulies iii ghoulies go to college,Fantasy Drama Horror Action Comedy
21985,negative happy chain saw edge,Fantasy Drama Horror Action Comedy Romance For...
11487,firestarter,Fantasy Drama Horror Action Thriller
10582,erotic ghost story,Fantasy Drama Horror
21960,nazareno cruz and the wolf,Fantasy Drama Horror
40599,waxworks,Fantasy Drama Horror
20598,midnight son,Drama Horror Action
4965,blood donuts,Horror Action Comedy
8038,curse of the wolf,Horror Action Comedy
8569,dead snow 2 red vs dead,Horror Action Comedy


In [204]:
#I called movies realted to Toy story and pops up related titles that a user can prefer. 

genre_recommendations('1000 on the black')

,title,genres
74,10000 dollars for a massacre,Western
415,40 guns to apache pass,Western
518,7th cavalry,Western
611,a big hand for the little lady,Western
654,a bullet for a stranger,Western
793,a dollar between the teeth,Western
833,a fistful of dollars,Western
917,a gun for one hundred graves,Western
1048,a man called blade,Western
1049,a man called gannon,Western


In [206]:
#I called movies realted to Toy story and pops up related titles that a user can prefer. 

genre_recommendations('aliens on the moon the truth exposed')

,title,genres
2,chicagogirl,Documentary
9,ellebrity,Documentary
18,til madness do us part,Documentary
23,asexual,Documentary
25,dishonesty the truth about lies,Documentary
32,and the pursuit of happiness,Documentary
44,1,Documentary
50,1 pm,Documentary
56,10 billion whats on your plate,Documentary
62,10 mph,Documentary


In [2]:
pickle_in = open("data_movie.pickle","rb")
data = pickle.load(pickle_in)

In [3]:
data.head(50)

,title,original_title,tagline,overview,genres,index
0,women art revolution,women art revolution,years making,intimate interviews provocative art rare histo...,Documentary,0
1,1 cheerleader camp,1 cheerleader camp,grab spirit sticks,pair horny college guys get summer jobs sexy c...,Comedy Drama,1
2,chicagogirl,chicagogirl,social network takes dictator,childhood bedroom chicago suburbs american tee...,Documentary,2
3,horror,horror,death trending,inspired actual events group year old girls ni...,Drama Mystery Horror Thriller,3
4,1000 on the black,mille dollari sul nero,never provided,johnny liston released prison wrongly served t...,Western,4
5,100000 for ringo,centomila dollari per ringo,never provided,stranger rides rainbow valley hes mistaken for...,Drama Western,5
6,5 a day,5 a day,relationships dont cheap,released jail joins father road,Drama Comedy,6
7,50k and a call girl a love story,50k and a call girl a love story,never provided,ross diagnosed terminal brain cancer given six...,Drama,7
8,999,999,never provided,ever wondered meaning life exist answer vexing...,Animation Drama,8
9,ellebrity,ellebrity,never provided,fame today obsession fame become millions us f...,Documentary,9


In [91]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(data['title'])

In [92]:
#Cosine similarity - using this I can create a kernel. 
# Using cosine similarity, I can get related or close feautes/items/movies as recommendations. 
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [119]:
# Build a 1-dimensional array with movie titles
titles = data['genres']
indices = pd.Series(data.index, index=data['title'])

# Function that get movie recommendations based on the cosine similarity score of movie genres
def genre_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[0:1]
    movie_indices = [i[0] for i in sim_scores]
    return titles[movie_indices]

In [120]:
print(genre_recommendations('toy story'))

38993    Animation Comedy Family
Name: genres, dtype: object


In [121]:
genre_recommendations('women art revolution')

0    Documentary
Name: genres, dtype: object

In [124]:
genre_recommendations('1 nenokkadine')

45    Action Comedy Romance
Name: genres, dtype: object

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf_overview = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english', dtype = 'uint8')
tfidf_matrix_overview =  tf_overview.fit_transform(data['overview'])

C:\Users\Vinay\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1817: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. uint8 'dtype' will be converted to np.float64.
  UserWarning)


In [26]:
tfidf_matrix_overview.shape

(42228, 976857)

In [27]:
tf_overview.get_feature_names()

['aa',
 'aa alcoholic',
 'aa chapter',
 'aa grew',
 'aa group',
 'aa meeting',
 'aa process',
 'aa tennis',
 'aaa',
 'aaa help',
 'aaaron',
 'aaaron kansas',
 'aabgamma',
 'aabgamma dispatched',
 'aachan',
 'aachan okamura',
 'aachan returns',
 'aachans',
 'aachans father',
 'aachans mother',
 'aachi',
 'aachi ssipak',
 'aackerlund',
 'aackerlund historic',
 'aadhavan',
 'aadhavan tamil',
 'aadland',
 'aadland highflying',
 'aaicha',
 'aaicha gho',
 'aakash',
 'aakash determined',
 'aakash past',
 'aakash singer',
 'aake',
 'aake doctors',
 'aake frightened',
 'aake world',
 'aalavandhan',
 'aalavandhan action',
 'aalst',
 'aalst textile',
 'aames',
 'aames tom',
 'aamir',
 'aamir khan',
 'aamir khans',
 'aampas',
 'aampas bars',
 'aampe',
 'aampe departments',
 'aampe network',
 'aampe production',
 'aampe tom',
 'aampes',
 'aampes jerusalem',
 'aampm',
 'aampm introduction',
 'aampr',
 'aampr feels',
 'aanandam',
 'aanandam indian',
 'aang',
 'aang young',
 'aangehouden',
 'aangehoud